##### it is an copy of my google collab file


##### please run it there it needed and download the newly trained model


 ##### i would also provide with the dataset used in the training the model

##### the computation power required for vectorization of the words is high

In [2]:
import fasttext

fasttext.FastText.eprint = lambda x: None

In [85]:
import fasttext

# Download the model (assuming it's compressed)
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

# Decompress the model file
!gzip -d cc.en.300.bin.gz

# Load the FastText model
model = fasttext.load_model('cc.en.300.bin')


--2024-01-25 22:13:24--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.251.114, 52.84.251.106, 52.84.251.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.251.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz.1’

cc.en.300.bin.gz.1  100%[===================>]   4.19G   173MB/s    in 27s     

2024-01-25 22:13:51 (159 MB/s) - ‘cc.en.300.bin.gz.1’ saved [4503593528/4503593528]



In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [88]:
df_fake = pd.read_csv("/content/drive/MyDrive/my dataset_training/project/Fake.csv")
df_true = pd.read_csv("/content/drive/MyDrive/my dataset_training/project/True.csv")

In [89]:
df_fake["class"] = 0
df_true["class"] = 1

In [90]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis = 0, inplace = True)


df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [91]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

<ipython-input-91-3aaf8ec2aad1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
<ipython-input-91-3aaf8ec2aad1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [92]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.shape

(44878, 5)

In [93]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [94]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [95]:
df = df.sample(frac = 1)

In [96]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [97]:
df.columns

Index(['text', 'class'], dtype='object')

In [98]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [99]:
df["text"] = df["text"].apply(wordopt)

In [100]:
x = df["text"]
y = df["class"]

In [101]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [105]:
X_train_embeddings = [model[text] for text in x_train]
# X_test_embeddings = [model[text] for text in test_df['text']]

In [106]:
from sklearn.linear_model import LogisticRegression

In [108]:
classifier = LogisticRegression()
classifier.fit(X_train_embeddings, y_train)

LogisticRegression()

In [110]:
X_test_embeddings = [model[text] for text in x_test]

In [111]:
predictions = classifier.predict(X_test_embeddings)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 85.04%
